In [12]:
import utilitiesHSC
import importlib
importlib.reload(utilitiesHSC)




<module 'utilitiesHSC' from 'C:\\Users\\picar\\Documents\\GitHub\\RPi-HeatingSys\\utilitiesHSC.py'>

In [ ]:
%run funcHSC.py


In [9]:
#!/usr/bin/env python3

# Control logic for the central heating system, run every 2 minutes
#   Import temperature data from csv (created by tempRead.py)
#   Import control logic from csv
#   Assign relay control to csv (used by valveControl.py)

# Import main modules
import time
import numpy as np
import pandas as pd
import os, sys, traceback
# Import code functions
from utilitiesHSC import getSetpointTemp
from utilitiesHSC import tryReadCSV
from utilitiesHSC import tryReadCSV_p

# Initialization
file_tempSensor="../RPi-HeatingSys-Data/dataTempSensor.csv"
file_tempSetpoint="../RPi-HeatingSys-Data/tempSetpoint.csv"
file_valveCmd="../RPi-HeatingSys-Data/valveCmd.csv"
file_tempWeather="../RPi-HeatingSys-Data/dataTempWeather.csv"
file_holdSetpoint="../RPi-HeatingSys-Data/holdCmd.csv"
lastDateTime=pd.to_datetime('today')
exitFlag=False

# Setup
typeDayRef=['Week','Week','Week','Week','Week','WeekEnd','WeekEnd']
typeZone=['Upstair','Main']
valveName=['V1U','V2M']

print('[%.19s] funcHSC.py: Setup completed, starting control' % pd.to_datetime('today'))

[2021-01-01 13:22:52] funcHSC.py: Setup completed, starting control


In [10]:
read_tempSensor,errorActive=tryReadCSV(file_tempSensor,'DateTime ()',pd)
display(read_tempSensor)
display(errorActive)

,TA_U (C),TF_U (C),TA_M (C),TF_M (C),TW_IN (C),TW_OUT (C),TA_UD (C),HA_UD (%),TA_MD (C),HA_MD (%)
DateTime (),,,,,,,,,,
2020-12-31 20:35:36.726493,22.2500,24.0000,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:35:54.806491,22.2500,24.0000,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:36:12.886490,22.2500,23.9375,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:36:30.965941,22.2500,23.9375,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:36:49.085888,22.2500,24.0000,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-12-31 21:03:38.086610,22.3125,23.8125,22.3750,22.8125,23.0625,20.500,NaN,NaN,NaN,NaN
2020-12-31 21:03:56.166238,22.3125,23.8125,22.3750,22.8125,23.0000,20.500,NaN,NaN,NaN,NaN
2020-12-31 21:04:14.246556,22.3125,23.8125,22.3750,22.8125,23.0000,20.500,NaN,NaN,NaN,NaN


False

In [15]:
read_tempSensor.index=pd.to_datetime(read_tempSensor.index)
print(read_tempSensor)

                            TA_U (C)  TF_U (C)  TA_M (C)  TF_M (C)  TW_IN (C)  \
DateTime ()                                                                     
2020-12-31 20:35:36.726493   22.2500   24.0000   22.3125   22.8750    23.6250   
2020-12-31 20:35:54.806491   22.2500   24.0000   22.3125   22.8750    23.6250   
2020-12-31 20:36:12.886490   22.2500   23.9375   22.3125   22.8750    23.6250   
2020-12-31 20:36:30.965941   22.2500   23.9375   22.3125   22.8750    23.6250   
2020-12-31 20:36:49.085888   22.2500   24.0000   22.3125   22.8750    23.6250   
...                              ...       ...       ...       ...        ...   
2020-12-31 21:03:38.086610   22.3125   23.8125   22.3750   22.8125    23.0625   
2020-12-31 21:03:56.166238   22.3125   23.8125   22.3750   22.8125    23.0000   
2020-12-31 21:04:14.246556   22.3125   23.8125   22.3750   22.8125    23.0000   
2020-12-31 21:04:32.326401   22.3125   23.8125   22.3750   22.8125    23.0625   
2020-12-31 21:04:50.407082  

In [21]:
read_tempSensor,errorActive=tryReadCSV_p(file_tempSensor,'DateTime ()',pd,attemptCount=5,parseCol='DateTime ()')
display(read_tempSensor)
display(read_tempSensor.index.dtype)

,TA_U (C),TF_U (C),TA_M (C),TF_M (C),TW_IN (C),TW_OUT (C),TA_UD (C),HA_UD (%),TA_MD (C),HA_MD (%)
DateTime (),,,,,,,,,,
2020-12-31 20:35:36.726493,22.2500,24.0000,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:35:54.806491,22.2500,24.0000,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:36:12.886490,22.2500,23.9375,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:36:30.965941,22.2500,23.9375,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
2020-12-31 20:36:49.085888,22.2500,24.0000,22.3125,22.8750,23.6250,20.875,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-12-31 21:03:38.086610,22.3125,23.8125,22.3750,22.8125,23.0625,20.500,NaN,NaN,NaN,NaN
2020-12-31 21:03:56.166238,22.3125,23.8125,22.3750,22.8125,23.0000,20.500,NaN,NaN,NaN,NaN
2020-12-31 21:04:14.246556,22.3125,23.8125,22.3750,22.8125,23.0000,20.500,NaN,NaN,NaN,NaN


dtype('<M8[ns]')

In [22]:
np.dtype('datetime64[ns]') == np.dtype('<M8[ns]')

True